In [3]:
pip install transformers torch requests beautifulsoup4 pandas nltk feedparser tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import feedparser
import pandas as pd
from transformers import pipeline
from datetime import datetime

# ─── CONFIG ────────────────────────────────────────────────────────────────
STOCK       = "TITAN"    # stock ticker or name to search
MAX_HEAD    = 50           # how many headlines to fetch
MODEL_NAME  = "ProsusAI/finbert"
# ────────────────────────────────────────────────────────────────────────────

# 1) Fetch Google News RSS for the stock
def fetch_google_news_headlines(query, max_items=10):
    """
    Uses Google News RSS to return the top headlines for a search query.
    """
    # Google News RSS URL for search:
    url = f"https://news.google.com/rss/search?q={query}+stock+india&hl=en-IN&gl=IN&ceid=IN:en"
    feed = feedparser.parse(url)
    headlines = []
    for entry in feed.entries[:max_items]:
        headlines.append(entry.title)
    return headlines

# 2) Load the FinBERT sentiment-analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME
)

# 3) Score each headline
def score_headlines(headlines):
    """
    Returns a DataFrame with [timestamp, headline, label, score].
    """
    results = sentiment_model(headlines)
    df = pd.DataFrame(results)
    df["headline"]  = headlines
    df["timestamp"] = datetime.now()
    # Ensure consistent column names
    df = df.rename(columns={
        df.columns[0]:"label",
        df.columns[1]:"score"
    })
    return df[["timestamp","headline","label","score"]]

# 4) Aggregate into a single weighted score
def aggregate_score(df):
    """
    Map labels to numerical values and compute a weighted mean.
    """
    mapping = {"positive":1, "neutral":0, "negative":-1}
    df["numeric"] = df["label"].map(mapping) * df["score"]
    return df["numeric"].mean()

# 5) End-to-end
def sentiment_bot(stock):
    print(f"\n🔎 Fetching top {MAX_HEAD} headlines for '{stock}' from Google News…")
    headlines = fetch_google_news_headlines(stock, max_items=MAX_HEAD)
    if not headlines:
        print("⚠️  No headlines found.")
        return
    
    print("📰 Headlines:")
    for i, h in enumerate(headlines, 1):
        print(f"  {i}. {h}")
    
    df_scores = score_headlines(headlines)
    avg = aggregate_score(df_scores)
    
    print("\n💬 Sentiment scores:")
    print(df_scores.to_string(index=False))
    print(f"\n👉 Average weighted sentiment: {avg:.3f}")
    
    if avg >  0.05:
        print("Overall sentiment: 📈 POSITIVE → consider LONG bias")
    elif avg < -0.05:
        print("Overall sentiment: 📉 NEGATIVE → consider SHORT bias")
    else:
        print("Overall sentiment: ➖ NEUTRAL")

if __name__ == "__main__":
    sentiment_bot(STOCK)


Device set to use mps:0



🔎 Fetching top 50 headlines for 'TITAN' from Google News…
📰 Headlines:
  1. Titan shares give Rs 900 crore shock to Jhunjhunwalas. What brokerages say on Tata’s bluechip stock - The Economic Times
  2. Titan shares decline 5% even as analysts remain optimistic on stock after Q1 update - CNBC TV18
  3. Titan slips hard: Shares slump 5.5% after weak Q1 update; Rs 900 crore knocked off Jhunjhunwalas’ portfol - The Times of India
  4. Titan share price falls 5% after Q1 business update. Time to buy, sell or hold? - Mint
  5. Stocks to Watch today, Tuesday, July 8: Tata Motors, Titan, NLC india - Business Standard
  6. Titan Share Price Today: Jewellery stock falls nearly 6% post Q1 update - Details here - India TV News
  7. Titan shares tumble 5% after Q1 business update fails to impress D-Street - The Economic Times
  8. Stock Market Updates: Sensex Dips 50 Points, Nifty Below 25,500; SMIDs In Green; Titan Slides 4% - News18
  9. Stocks to watch: Tata Motors, Titan, M&M, among shares in 

In [4]:
import feedparser
import pandas as pd
from transformers import pipeline
from datetime import datetime

# ─── CONFIG ────────────────────────────────────────────────────────────────
# List of Nifty 50 tickers (Yahoo Finance format)
nifty50 = [
    'ADANIPORTS.NS','ASIANPAINT.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BAJFINANCE.NS',
    'BAJAJFINSV.NS','BPCL.NS','BHARTIARTL.NS','BRITANNIA.NS','CIPLA.NS',
    'COALINDIA.NS','DIVISLAB.NS','DRREDDY.NS','EICHERMOT.NS','GAIL.NS',
    'GRASIM.NS','HDFCBANK.NS','HINDALCO.NS','HINDUNILVR.NS','HDFC.NS',
    'HCLTECH.NS','ICICIBANK.NS','INDUSINDBK.NS','INFY.NS','ITC.NS',
    'JSWSTEEL.NS','KOTAKBANK.NS','LT.NS','M&M.NS','MARUTI.NS',
    'NESTLEIND.NS','NTPC.NS','ONGC.NS','POWERGRID.NS','SBILIFE.NS',
    'SBIN.NS','SUNPHARMA.NS','TATAMOTORS.NS','TATASTEEL.NS','TCS.NS',
    'TECHM.NS','TITAN.NS','ULTRACEMCO.NS','UPL.NS','WIPRO.NS'
]
MAX_HEADLINES = 10
MODEL_NAME = "ProsusAI/finbert"
# ────────────────────────────────────────────────────────────────────────────

# Load FinBERT sentiment-analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME
)

def fetch_google_news_headlines(query, max_items=MAX_HEADLINES):
    """
    Fetch top headlines for a query via Google News RSS.
    """
    url = f"https://news.google.com/rss/search?q={query}+stock+india&hl=en-IN&gl=IN&ceid=IN:en"
    feed = feedparser.parse(url)
    return [entry.title for entry in feed.entries[:max_items]]

def score_headlines(headlines):
    """
    Score headlines using FinBERT transformer.
    Returns DataFrame with columns [timestamp, headline, label, score].
    """
    if not headlines:
        return pd.DataFrame(columns=["timestamp","headline","label","score"])
    results = sentiment_model(headlines)
    df = pd.DataFrame(results)
    df["headline"] = headlines
    df["timestamp"] = datetime.now()
    # rename columns to standard names
    df = df.rename(columns={df.columns[0]:"label", df.columns[1]:"score"})
    return df[["timestamp","headline","label","score"]]

def aggregate_score(df_scores):
    """
    Compute average weighted sentiment: positive=1, neutral=0, negative=-1 times score.
    """
    if df_scores.empty:
        return 0.0
    mapping = {"positive":1, "neutral":0, "negative":-1}
    df_scores["numeric"] = df_scores["label"].map(mapping) * df_scores["score"]
    return df_scores["numeric"].mean()

# Main loop: process all Nifty 50
records = []
for ticker in nifty50:
    query = ticker.split(".")[0]
    headlines = fetch_google_news_headlines(query)
    df_scores = score_headlines(headlines)
    avg = aggregate_score(df_scores)
    bias = "LONG" if avg > 0.05 else "SHORT" if avg < -0.05 else "NEUTRAL"
    records.append({
        "symbol": ticker,
        "avg_score": avg,
        "bias": bias,
        "num_headlines": len(headlines)
    })

# Create final DataFrame
result_df = pd.DataFrame(records)
result_df


Device set to use mps:0


,symbol,avg_score,bias,num_headlines
0,ADANIPORTS.NS,-0.254999,SHORT,10
1,ASIANPAINT.NS,-0.028280,NEUTRAL,10
2,AXISBANK.NS,-0.232779,SHORT,10
3,BAJAJ-AUTO.NS,-0.382970,SHORT,10
4,BAJFINANCE.NS,-0.067338,SHORT,10
5,BAJAJFINSV.NS,-0.269817,SHORT,10
6,BPCL.NS,-0.035499,NEUTRAL,10
7,BHARTIARTL.NS,0.077928,LONG,10
8,BRITANNIA.NS,-0.029407,NEUTRAL,10
9,CIPLA.NS,0.439261,LONG,10
